In [8]:
import theano
import theano.tensor as T
import numpy as np

X = T.matrix("X")
W = T.matrix("W")
b_sym = T.vector("b_sym")

results, updates = theano.scan(lambda v: T.tanh(T.dot(v, W) + b_sym), sequences=X)
compute_elementwise = theano.function(inputs=[X, W, b_sym], outputs=results)

x = np.eye(2, dtype=theano.config.floatX) # I (2x2)
w = np.ones((2,2), dtype=theano.config.floatX) # [[1,1],[1,1]]
b = np.ones((2), dtype=theano.config.floatX)
b[1] = 2 # [1,2]
print compute_elementwise(x,w,b)[0]

print np.tanh(x.dot(w)+b)
print np.tanh(2), np.tanh(3)

[ 0.96402758  0.99505475]
[[ 0.96402758  0.99505475]
 [ 0.96402758  0.99505475]]
0.964027580076 0.995054753687


In [11]:
X = T.vector("X")
W = T.matrix("W")
b_sym = T.vector("b_sym")

U = T.matrices("U")
Y = T.matrices("Y")
V = T.matrices("V")
P = T.matrices("P")

results, updates = theano.scan(lambda y, p, x_tm1: T.tanh(T.dot(x_tm1, W) + T.dot(y, U) + T.dot(p, V)),
                              sequences=[Y, P[::-1]], outputs_info=[X])
compute_seq = theano.function(inputs=[X, W, Y, U, P, V], outputs=[results])

x = np.zeros((2), dtype=theano.config.floatX)
x[1] = 1
w = np.zeros((2,2), dtype=theano.config.floatX)
y= np.zeros((5,2), dtype=theano.config.floatX)
y[0,:] = -3
u = np.ones((2,2), dtype=theano.config.floatX)
p = np.ones((5,2), dtype=theano.config.floatX)
p[0,:] = 3
v = np.ones((2,2), dtype=theano.config.floatX)

print compute_seq(x, w, y, u, p, v)[0]

# comparison with numpy
x_res = np.zeros((5, 2), dtype=theano.config.floatX)
x_res[0] = np.tanh(x.dot(w) + y[0].dot(u) + p[4].dot(v))
for i in range(1, 5):
  x_res[i] = np.tanh(x_res[i - 1].dot(w) + y[i].dot(u) + p[4-i].dot(v))
print x_res

[[-0.9993293  -0.9993293 ]
 [ 0.96402758  0.96402758]
 [ 0.96402758  0.96402758]
 [ 0.96402758  0.96402758]
 [ 0.99998771  0.99998771]]
[[-0.9993293  -0.9993293 ]
 [ 0.96402758  0.96402758]
 [ 0.96402758  0.96402758]
 [ 0.96402758  0.96402758]
 [ 0.99998771  0.99998771]]


In [30]:
# factorial
import math
X = T.vector("x")
time = T.vector("t")

results, updates = theano.scan(lambda t, x: (t+1) * x, sequences=[time], outputs_info=[X])
my_loop = theano.function(inputs=[X,time], outputs=[results])

t = np.array([1,2,3,4,5,6,7,8,9,10], dtype=theano.config.floatX)
print my_loop(np.ones(1, dtype=theano.config.floatX), t)
for i in range(2, 12):
    print math.factorial(i)

[array([[  2.00000000e+00],
       [  6.00000000e+00],
       [  2.40000000e+01],
       [  1.20000000e+02],
       [  7.20000000e+02],
       [  5.04000000e+03],
       [  4.03200000e+04],
       [  3.62880000e+05],
       [  3.62880000e+06],
       [  3.99168000e+07]])]
2
6
24
120
720
5040
40320
362880
3628800
39916800


In [31]:
X = T.matrix("X")
results, updates = theano.scan(lambda x_i: T.sqrt((x_i **  2).sum()), sequences=[X])
compute_norm_lines = theano.function(inputs=[X], outputs=[results])

x = np.diag(np.arange(1, 6, dtype=theano.config.floatX), 1)
print compute_norm_lines(x)[0]

print np.sqrt((x ** 2).sum(1))

[ 1.  2.  3.  4.  5.  0.]
[ 1.  2.  3.  4.  5.  0.]


In [32]:
results, updates = theano.scan(lambda x_i: T.sqrt((x_i **  2).sum()), sequences=[X.T])
compute_norm_cols = theano.function(inputs=[X], outputs=[results])

x = np.diag(np.arange(1, 6, dtype=theano.config.floatX), 1)
print compute_norm_cols(x)[0]

print np.sqrt((x ** 2).sum(0))

[ 0.  1.  2.  3.  4.  5.]
[ 0.  1.  2.  3.  4.  5.]


In [37]:
X = T.matrix("X")
floatX = "float32"
results, updates = theano.scan(lambda i,j,t_f: T.cast(X[i,j] + t_f, floatX),
                              sequences=[T.arange(X.shape[0]), T.arange(X.shape[1])],
                              outputs_info=np.asarray(0., dtype=floatX))
result = results[-1]
compute_trace = theano.function(inputs=[X], outputs=[result])

x = np.eye(5, dtype=theano.config.floatX)
x[0] = np.arange(5., dtype=theano.config.floatX)
print x
print compute_trace(x)[0]

print np.diagonal(x).sum()

[[ 0.  1.  2.  3.  4.]
 [ 0.  1.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.]
 [ 0.  0.  0.  1.  0.]
 [ 0.  0.  0.  0.  1.]]
4.0
4.0


In [40]:
# use x(t-2)
X = T.matrix("X")
n_sym = T.iscalar("n_sym")
results, updates = theano.scan(lambda x_tm2, x_tm1: x_tm2 + x_tm1,
                                n_steps=n_sym, outputs_info=[dict(initial=X, taps=[-2,-1])])
compute_seq2 = theano.function(inputs=[X,n_sym], outputs=[results])
x = np.ones((2,1), dtype=theano.config.floatX)
n = 10

print compute_seq2(x,n)

[array([[   2.],
       [   3.],
       [   5.],
       [   8.],
       [  13.],
       [  21.],
       [  34.],
       [  55.],
       [  89.],
       [ 144.]])]


In [46]:
v = T.vector()
A = T.matrix()
y = T.tanh(T.dot(v, A))
results, updates = theano.scan(lambda i: T.grad(y[i], v), sequences=[T.arange(y.shape[0])])
compute_jac_t = theano.function([A, v], [results], allow_input_downcast=True) # shape (d_out, d_in)

# test values
x = np.ones((1,5), dtype=theano.config.floatX)[0]
w = np.eye(5, 3, dtype=theano.config.floatX)
w[2] = np.ones((3), dtype=theano.config.floatX)
print compute_jac_t(w, x)[0]

# compare with numpy
print x
print w
print ((1 - np.tanh(x.dot(w)) ** 2) * w).T

[[ 0.07065082  0.          0.07065082  0.          0.        ]
 [ 0.          0.07065082  0.07065082  0.          0.        ]
 [ 0.          0.          0.41997434  0.          0.        ]]
[ 1.  1.  1.  1.  1.]
[[ 1.  0.  0.]
 [ 0.  1.  0.]
 [ 1.  1.  1.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]]
[[ 0.07065082  0.          0.07065082  0.          0.        ]
 [ 0.          0.07065082  0.07065082  0.          0.        ]
 [ 0.          0.          0.41997434  0.          0.        ]]


In [50]:
k = theano.shared(0)
n_sym = T.iscalar("n_sym")

results, updates = theano.scan(lambda :{k:(k + 1)}, n_steps=n_sym)
accumulator = theano.function([n_sym], [], updates=updates, allow_input_downcast=True)

print k.get_value()
accumulator(5)
print k.get_value()

0
5


In [57]:
k = theano.shared(0.)
n_sym = T.vector("n_sym")

results, updates = theano.scan(lambda i:{k:(k + i)}, sequences=n_sym)
accumulator = theano.function([n_sym], [], updates=updates, allow_input_downcast=True)

print k.get_value()
print np.arange(5)
accumulator(np.arange(5))
print k.get_value()

0.0
[0 1 2 3 4]
10.0


In [58]:
# define tensor variables
X = T.matrix("X")
W = T.matrix("W")
b_sym = T.vector("b_sym")

# define shared random stream
trng = T.shared_randomstreams.RandomStreams(1234)
d=trng.binomial(size=W[1].shape)

results, updates = theano.scan(lambda v: T.tanh(T.dot(v, W) + b_sym) * d, sequences=X)
compute_with_bnoise = theano.function(inputs=[X, W, b_sym], outputs=[results],
                          updates=updates, allow_input_downcast=True)
x = np.eye(10, 2, dtype=theano.config.floatX)
w = np.ones((2, 2), dtype=theano.config.floatX)
b = np.ones((2), dtype=theano.config.floatX)

print compute_with_bnoise(x, w, b)

[array([[ 0.96402758,  0.        ],
       [ 0.        ,  0.96402758],
       [ 0.        ,  0.        ],
       [ 0.76159416,  0.76159416],
       [ 0.76159416,  0.        ],
       [ 0.        ,  0.76159416],
       [ 0.        ,  0.76159416],
       [ 0.        ,  0.76159416],
       [ 0.        ,  0.        ],
       [ 0.76159416,  0.76159416]])]


In [59]:
theano.config.warn.subtensor_merge_bug = False

k = T.iscalar("k")
A = T.vector("A")

def inner_fct(prior_result, B):
    return prior_result * B

# Symbolic description of the result
result, updates = theano.scan(fn=inner_fct,
                            outputs_info=T.ones_like(A),
                            non_sequences=A, n_steps=k)

# Scan has provided us with A ** 1 through A ** k.  Keep only the last
# value. Scan notices this and does not waste memory saving them.
final_result = result[-1]

power = theano.function(inputs=[A, k], outputs=final_result,
                      updates=updates)

print power(range(10), 2)
#[  0.   1.   4.   9.  16.  25.  36.  49.  64.  81.]

[  0.   1.   4.   9.  16.  25.  36.  49.  64.  81.]


In [86]:
coefficients = theano.tensor.vector("coefficients")
x = T.scalar("x")
#rslt = T.scalar("r")
rslt = T.scalar("r")
#rslt = T.as_tensor_variable(np.asarray(0, 'float64'))
max_coefficients_supported = 10000

# Generate the components of the polynomial
full_range=theano.tensor.arange(max_coefficients_supported)
components, updates = theano.scan(fn=lambda coeff, power, acc, free_var:
                                acc + ( coeff * (free_var ** power)),
                                outputs_info=[rslt],
                                sequences=[coefficients, full_range],
                                non_sequences=x)

calculate_polynomial = theano.function(inputs=[coefficients, x, rslt],
                                     outputs=components[-1])

test_coeff = np.asarray([1, 0, 2], dtype=np.float32)
print calculate_polynomial(test_coeff, 3, 0.)
# 19.0

19.0
